<a href="https://colab.research.google.com/github/jeevisha19/DATA-SCIENCE/blob/main/Transformer_Chatbot_Version1_NoRAG_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers torch nltk


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-medium")
model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-medium")

print("Transformer chatbot brain loaded successfully!")


pytorch_model.bin:   0%|          | 0.00/863M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/863M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Transformer chatbot brain loaded successfully!


In [ ]:
import torch

chat_history_ids = None

print("Start chatting with the bot (type 'bye' to stop)")

while True:
  user_input = input("You: ")
  if user_input.lower() == "bye":
    print("Bot: Goodbye!")
    break

  new_input_ids = tokenizer.encode(
      user_input + tokenizer.eos_token, return_tensors='pt'
  )

  # Fix: Corrected conditional assignment for bot_input_ids
  bot_input_ids = torch.cat([chat_history_ids, new_input_ids], dim=1) if chat_history_ids is not None else new_input_ids

  chat_history_ids = model.generate(
      bot_input_ids,
      max_length=1000,
      pad_token_id=tokenizer.eos_token_id
  )

  # Fix: Corrected slicing for reply to get a 1D tensor for decoding
  reply = tokenizer.decode(
      chat_history_ids[0, bot_input_ids.shape[-1]:],
      skip_special_tokens=True
  )

  print("Bot:", reply)

Start chatting with the bot (type 'bye' to stop)
Bot: Hello! :D
Bot: I'm good, how are you?
Bot: I'm good too!


In [ ]:
questions=[
    "What is Machine Learning?",
    "What is NLP?",
    "Explain Artificial Intelligence",
    "What is Deep Learning",
    "What is is a Transfromer model?"
    ]

In [ ]:
references = [
    "Machine learning is a subset of artificial intelligence that allows systems to learn from data.",
    "Natural Language Processing is a field of AI that helps machines understand human language.",
    "Artificial intelligence is the simulation of human intelligence by machines.",
    "Deep learning is a subset of machine learning that uses neural networks with many layers.",
    "A transformer model is a deep learning model based on self-attention mechanisms."
]


In [ ]:
bot_answers=[]
for q in questions:
  input_ids = tokenizer.encode(q +tokenizer.eos_token, return_tensors='pt')
  output_ids = model.generate(
      input_ids,
      max_length=100,
      pad_token_id=tokenizer.eos_token_id

  )

  answer=tokenizer.decode(output_ids[0], skip_special_tokens=True)
  bot_answers.append(answer)

In [ ]:
!pip install nltk

In [ ]:
from nltk.translate.bleu_score import sentence_bleu

bleu_scores=[]

for ref, hyp in zip(references, bot_answers):
  score= sentence_bleu([ref.split()], hyp.split())
  bleu_scores.append(score)

  average_bleu= sum(bleu_scores)/ len(bleu_scores)
  print("Average BLEU Score:", average_bleu)

Average BLEU Score: 0.12607162121844206


In [ ]:
import torch
import math

def calculate_perplexity(text):
    encodings = tokenizer(text, return_tensors="pt")
    input_ids = encodings["input_ids"]

    with torch.no_grad():
        outputs = model(input_ids)
        logits = outputs.logits

    shift_logits = logits[:, :-1, :].contiguous()
    shift_labels = input_ids[:, 1:].contiguous()

    loss_fct = torch.nn.CrossEntropyLoss()
    loss = loss_fct(
        shift_logits.view(-1, shift_logits.size(-1)),
        shift_labels.view(-1)
    )

    perplexity = math.exp(loss.item())
    return perplexity

perplexities = [calculate_perplexity(ans) for ans in bot_answers]
avg_perplexity = sum(perplexities) / len(perplexities)

print("Average Perplexity:", avg_perplexity)


Average Perplexity: 109.77276082335275
